In [44]:
import pandas as pd
import numpy as np
import os
import sys
import unicodedata
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor

# --- 1. CONFIGURAÇÕES INICIAIS ---
FILE_INPUT = 'dados_ml_pronto.csv'
TEST_SIZE_WEEKS = 15 # Últimas X semanas para teste/validação
NUM_FUTURE_WEEKS = 12 # Número de semanas futuras para prever

# Dicionário de colunas opcionais e seus valores padrão
optional_cols = {
    'decoracao': 0, 'instagram': 0, 'panfletos': 0, 'feriado': 0,
    'pessoafisica': 0, 'empresa': 0, 'custo_total_campanhas': 0.0, 'preco_medio_venda': 0.0
}

# Início da execução
print("--- Iniciando o Processo de Previsão de Demanda (Etapa 1: Configurações Iniciais) ---")

--- Iniciando o Processo de Previsão de Demanda (Etapa 1: Configurações Iniciais) ---


In [45]:
# --- 2. FUNÇÕES AUXILIARES ---

def normalize_col_names(df):
    """Normaliza nomes de colunas para minúsculas e snake_case."""
    new_cols = []
    for col in df.columns:
        normalized_col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8').lower()
        new_cols.append(normalized_col.replace(' ', '_').replace('.', '').replace('/', '_').replace('(', '').replace(')', ''))
    df.columns = new_cols
    return df

def create_time_features(df_input):
    """Cria features de séries temporais (ano, mês, semana, lags, médias móveis)."""
    df_features = df_input.copy()

    # Features de Data
    df_features['Ano'] = df_features.index.year
    df_features['Mes'] = df_features.index.month
    df_features['DiaDaSemana'] = df_features.index.dayofweek
    df_features['SemanaDoAno'] = df_features.index.isocalendar().week.astype(int)

    # Features de Lag e Média Móvel (aplicadas apenas à 'quantidade_real')
    for i in range(1, 4):
        df_features[f'Lag_Quantidade_{i}S'] = df_features['quantidade_real'].shift(i)
    
    df_features['MediaMovel_Quantidade_4S'] = df_features['quantidade_real'].rolling(window=4, min_periods=1).mean()
    df_features['MediaMovel_Quantidade_8S'] = df_features['quantidade_real'].rolling(window=8, min_periods=1).mean()

    # Feature 'SemanasSinceLastSale'
    df_features.loc[:, 'SemanasSinceLastSale'] = 0
    weeks_since_last = 0
    for idx, row in df_features.iterrows():
        if row['quantidade_real'] > 0:
            weeks_since_last = 0
        else:
            weeks_since_last += 1
        df_features.loc[idx, 'SemanasSinceLastSale'] = weeks_since_last
    
    df_features['VendaNaSemanaAnterior'] = (df_features['Lag_Quantidade_1S'] > 0).astype(int)

    # Preenche NaNs resultantes dos lags iniciais (primeiras linhas) com 0
    df_features.fillna(0, inplace=True) 
    return df_features

print("--- Funções auxiliares definidas. ---")

--- Funções auxiliares definidas. ---


In [46]:
# --- 3. CARREGAMENTO E PRÉ-PROCESSAMENTO INICIAL DOS DADOS ---
print("Carregando e pré-processando o arquivo de entrada...")
try:
    df_raw = pd.read_csv(FILE_INPUT, sep=',', decimal='.', parse_dates=['SemanaAno'], dayfirst=True)
    df_raw = normalize_col_names(df_raw)

    df_raw.rename(columns={'semanaano': 'Timestamp'}, inplace=True)
    df_raw.set_index('Timestamp', inplace=True)
    df_raw.sort_index(inplace=True)

    if 'quantidade' in df_raw.columns:
        df_raw.rename(columns={'quantidade': 'quantidade_real'}, inplace=True)
    elif 'quantidade_real' not in df_raw.columns:
        print("ERRO: Coluna de quantidade ('Quantidade' ou 'quantidade_real') não encontrada no CSV.")
        sys.exit(1)
    
    if 'produto' in df_raw.columns:
        df_raw.rename(columns={'produto': 'nome_produto'}, inplace=True)
    elif 'nome_produto' not in df_raw.columns:
        print("ERRO: Coluna 'Produto' ou 'nome_produto' não encontrada no CSV de entrada.")
        sys.exit(1)
    
    # Validação para um único produto
    if df_raw['nome_produto'].nunique() != 1:
        print(f"ERRO: O arquivo '{FILE_INPUT}' deve conter dados de APENAS UM PRODUTO. Encontrados: {df_raw['nome_produto'].unique()}")
        sys.exit(1)
    PRODUCT_NAME = df_raw['nome_produto'].iloc[0]

    df_raw['quantidade_real'] = pd.to_numeric(df_raw['quantidade_real'], errors='coerce').fillna(0).astype(int)

    if df_raw['quantidade_real'].sum() == 0:
        print(f"ALERTA CRÍTICO: A coluna 'quantidade_real' para o produto '{PRODUCT_NAME}' contém apenas zeros. Verifique seu CSV.")
        
    print(f"Arquivo '{FILE_INPUT}' carregado com sucesso para o produto: '{PRODUCT_NAME}'.")

except FileNotFoundError:
    print(f"ERRO: Arquivo '{FILE_INPUT}' não encontrado. Verifique o caminho e o nome do arquivo.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO inesperado ao carregar ou pré-processar o arquivo: {e}. Verifique o formato do CSV (separador, decimal) e nomes das colunas.")
    sys.exit(1)

Carregando e pré-processando o arquivo de entrada...
Arquivo 'dados_ml_pronto.csv' carregado com sucesso para o produto: 'Petisco Natural de Frango 500g'.


In [47]:
# --- 4. GARANTIR SÉRIES TEMPORAIS CONTÍNUAS E PADRONIZAR COLUNAS ADICIONAIS ---
print("Completando séries temporais com semanas ausentes e padronizando colunas...")

# Garante que o índice é um DatetimeIndex para operações de série temporal
if not isinstance(df_raw.index, pd.DatetimeIndex):
    df_raw.index = pd.to_datetime(df_raw.index)

start_date = df_raw.index.min()
end_date = df_raw.index.max()

# Ajusta o range de datas para cobrir semanas completas (segunda a domingo)
start_date_aligned = start_date - pd.Timedelta(days=start_date.dayofweek)
end_date_aligned = end_date + pd.Timedelta(days=(6 - end_date.dayofweek) if end_date.dayofweek != 6 else 0)

full_date_range = pd.date_range(start=start_date_aligned, end=end_date_aligned, freq='W-MON')

# Cria DataFrame processado, garantindo que os dados originais sejam preservados
df_product_processed = pd.DataFrame(index=full_date_range)
df_product_processed.index.name = 'Timestamp'
df_product_processed = df_product_processed.join(df_raw[['quantidade_real', 'nome_produto']], how='left')

# Preenche NaNs de quantidade_real e nome_produto para as semanas ausentes
df_product_processed['quantidade_real'] = df_product_processed['quantidade_real'].fillna(0).astype(int)
df_product_processed['nome_produto'] = df_product_processed['nome_produto'].fillna(PRODUCT_NAME)

# Tratamento de colunas de flags e custos
for col, default_val in optional_cols.items():
    if col in df_raw.columns:
        df_product_processed[col] = df_raw[col].reindex(df_product_processed.index).fillna(default_val)
        if col in ['decoracao', 'instagram', 'panfletos', 'feriado', 'pessoafisica', 'empresa']:
            df_product_processed[col] = df_product_processed[col].astype(int)
    else:
        df_product_processed[col] = default_val

# Renomeia 'custocampanha' se existir
if 'custocampanha' in df_product_processed.columns and 'custo_total_campanhas' not in df_product_processed.columns:
    df_product_processed.rename(columns={'custocampanha': 'custo_total_campanhas'}, inplace=True)

# Preenchimento forward-fill para 'preco_medio_venda' se houver dados, senão 0
if 'preco_medio_venda' in df_product_processed.columns:
    df_product_processed['preco_medio_venda'] = df_product_processed['preco_medio_venda'].fillna(method='ffill').fillna(0)
else:
    df_product_processed['preco_medio_venda'] = 0

df_product_processed = df_product_processed.sort_index()
print(f"Séries temporais completadas. Total de semanas para o produto: {len(df_product_processed)}.")

Completando séries temporais com semanas ausentes e padronizando colunas...
Séries temporais completadas. Total de semanas para o produto: 53.


C:\Users\Deni\AppData\Local\Temp\ipykernel_1052\3723250017.py:41: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_product_processed['preco_medio_venda'] = df_product_processed['preco_medio_venda'].fillna(method='ffill').fillna(0)


In [48]:
# --- 5. ENGENHARIA DE FEATURES ---
print("Criando features de séries temporais (lags, médias móveis e outras)...")
df_features = create_time_features(df_product_processed)

features_to_exclude = ['quantidade_real', 'nome_produto']
X = df_features.drop(columns=[col for col in features_to_exclude if col in df_features.columns], errors='ignore')
y = df_features['quantidade_real']

feature_columns = X.columns.tolist()

if X.empty or y.empty:
    print("ERRO: O DataFrame de features (X) ou o target (y) está vazio. Verifique os dados de entrada e processamento.")
    sys.exit(1)

print(f"Features criadas. Total de features para o modelo: {len(feature_columns)}.")

Criando features de séries temporais (lags, médias móveis e outras)...
Features criadas. Total de features para o modelo: 19.


In [49]:
# --- 6. DIVISÃO DOS DADOS EM TREINO E TESTE (TIME-SERIES SPLIT) ---
print(f"Dividindo dados em treino e teste (últimas {TEST_SIZE_WEEKS} semanas para teste)...")

if len(X) <= TEST_SIZE_WEEKS:
    print(f"AVISO: O número total de semanas ({len(X)}) é menor ou igual ao número de semanas para teste ({TEST_SIZE_WEEKS}). Ajustando TEST_SIZE_WEEKS para {len(X) - 1 if len(X) > 1 else 1}.")
    TEST_SIZE_WEEKS = max(1, len(X) - 1)

X_train = X.iloc[:-TEST_SIZE_WEEKS]
X_test = X.iloc[-TEST_SIZE_WEEKS:]
y_train = y.iloc[:-TEST_SIZE_WEEKS]
y_test = y.iloc[-TEST_SIZE_WEEKS:]

if X_train.empty or X_test.empty or y_train.empty or y_test.empty:
    print("ERRO: A divisão de treino/teste resultou em conjuntos vazios. Ajuste 'TEST_SIZE_WEEKS'.")
    sys.exit(1)

print(f"Período de Treino: {X_train.index.min().strftime('%d/%m/%Y')} a {X_train.index.max().strftime('%d/%m/%Y')} ({len(X_train)} semanas)")
print(f"Período de Teste:  {X_test.index.min().strftime('%d/%m/%Y')} a {X_test.index.max().strftime('%d/%m/%Y')} ({len(X_test)} semanas)")

if y_train.sum() == 0:
    print("ALERTA: O conjunto de treino não contém vendas (todos os valores são zero). O modelo aprenderá a prever zero.")
if y_test.sum() == 0:
    print("ALERTA: O conjunto de teste não contém vendas (todos os valores são zero). As métricas de avaliação serão 0 ou NaN.")

Dividindo dados em treino e teste (últimas 15 semanas para teste)...
Período de Treino: 01/01/2024 a 16/09/2024 (38 semanas)
Período de Teste:  23/09/2024 a 30/12/2024 (15 semanas)


In [50]:
# --- 7. TREINAMENTO DO MODELO XGBoost (Sem GridSearchCV) ---
print("\n" + "="*50)
print("     *** Treinando o Modelo XGBoost (Sem Otimização de Hiperparâmetros) *** ")
print("==================================================")

# Configurando o modelo XGBoost com parâmetros que tendem a dar bons resultados (padrão ou ligeiramente ajustados)
# Estes são os parâmetros que provavelmente estavam ativos quando você obteve o R² de 59%
model = XGBRegressor(objective='reg:squarederror', 
                     eval_metric='mae', 
                     random_state=42, 
                     n_jobs=-1,
                     n_estimators=100,  # Exemplo de um número razoável de estimadores
                     learning_rate=0.1, # Exemplo de uma taxa de aprendizado padrão
                     max_depth=3)      # Exemplo de uma profundidade de árvore padrão

# Treinando o modelo
model.fit(X_train, y_train)

print("Treinamento do modelo XGBoost concluído.")
print("==================================================")


     *** Treinando o Modelo XGBoost (Sem Otimização de Hiperparâmetros) *** 
Treinamento do modelo XGBoost concluído.


In [51]:
# --- 7. TREINAMENTO DO MODELO XGBoost (Ajuste para Forçar Variação Futura) ---
print("\n" + "="*50)
print("     *** Treinando o Modelo XGBoost (Com Ajustes para Variação) *** ")
print("==================================================")

# Tentando ajustar parâmetros para forçar mais variação nas previsões futuras
model = XGBRegressor(objective='reg:squarederror', 
                     eval_metric='mae', 
                     random_state=42, 
                     n_jobs=-1,
                     n_estimators=200,      # Aumentado para dar mais chance ao modelo
                     learning_rate=0.05,    # Diminuído para um aprendizado mais gradual
                     max_depth=5,           # Aumentado para permitir mais complexidade
                     min_child_weight=0.5,  # Reduzido para permitir divisões com menor peso (mais sensível a dados esparsos)
                     gamma=0.01,            # Pequena regularização, mas não muito alta
                     subsample=0.8,         # Usa 80% das amostras para cada árvore
                     colsample_bytree=0.8)  # Usa 80% das features para cada árvore

# Treinando o modelo
model.fit(X_train, y_train)

print("Treinamento do modelo XGBoost concluído com ajustes.")
print("==================================================")


     *** Treinando o Modelo XGBoost (Com Ajustes para Variação) *** 
Treinamento do modelo XGBoost concluído com ajustes.


In [52]:
# --- 8. AVALIAÇÃO DE DESEMPENHO DO MODELO ---
print("\n" + "="*50)
print("     *** RESULTADOS DAS MÉTRICAS (XGBoost Treinado) *** ")
print("==================================================")

# INICIALIZAÇÃO DE VARIÁVEIS (Adicionar estas 3 linhas)
r2 = np.nan
mae = np.nan
mape = np.nan

y_pred_test_raw = model.predict(X_test)
# Ajuste para prever zero e arredondar
y_pred_test = np.round(y_pred_test_raw).clip(min=0)
y_pred_test[y_pred_test < 0.5] = 0 # Define como 0 se a previsão for menor que 0.5
y_pred_test = y_pred_test.astype(int)

if not y_test.empty and y_pred_test.size > 0:
    y_test_original = y_test.values

    if np.std(y_test_original) == 0:
        r2 = np.nan
        print("AVISO: Desvio padrão dos valores reais no teste é zero. R-squared não calculado.")
    else:
        r2 = r2_score(y_test_original, y_pred_test)
        print(f"R-squared (R²): {r2:.4f}")

    mae = mean_absolute_error(y_test_original, y_pred_test)
    print(f"MAE (Mean Absolute Error): {mae:.2f}")

    mape = np.nan
    non_zero_indices = (y_test_original != 0)
    if np.sum(non_zero_indices) > 0:
        mape = np.mean(np.abs((y_test_original[non_zero_indices] - y_pred_test[non_zero_indices]) / y_test_original[non_zero_indices])) * 100
        print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
    else:
        print("MAPE: Não calculado (todos os valores reais no teste são zero).")
else:
    print("AVISO: Conjuntos de teste ou predições vazios para cálculo de métricas.")
print("==================================================")

print("\n--- Importância das Features (Top 15) ---")
if len(feature_columns) > 0:
    feature_importances = pd.Series(model.feature_importances_, index=feature_columns).sort_values(ascending=False)
    print(feature_importances.head(15))
else:
    print("Não há features para exibir a importância.")
print("==================================================")

# --- 8.1. EXPORTANDO DADOS DE TESTE (REAL vs. PREDITO) PARA CSV ---
print("\n" + "="*50)
print("     *** EXPORTANDO DADOS DE TESTE PARA CSV *** ")
print("==================================================")

# Criando um DataFrame com os resultados do teste
df_test_results = pd.DataFrame({
    'Timestamp': y_test.index.strftime('%d/%m/%Y'),
    'Produto': PRODUCT_NAME,
    'Real': y_test.values,
    'Predito': y_pred_test
})

# Definindo o nome do arquivo de saída
safe_product_name = unicodedata.normalize('NFKD', PRODUCT_NAME).encode('ascii', 'ignore').decode('utf-8')
safe_product_name = safe_product_name.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "")
test_output_filename = f'resultados_teste_xgboost_{safe_product_name}.csv'

# Exportando para CSV
df_test_results.to_csv(test_output_filename, index=False, encoding='utf-8-sig', sep=';')

print(f"Arquivo '{test_output_filename}' com resultados do teste exportado com sucesso!")
print("==================================================")


     *** RESULTADOS DAS MÉTRICAS (XGBoost Treinado) *** 
R-squared (R²): 0.7619
MAE (Mean Absolute Error): 0.67
MAPE (Mean Absolute Percentage Error): 17.82%

--- Importância das Features (Top 15) ---
pessoafisica                0.283383
empresa                     0.156970
panfletos                   0.089147
MediaMovel_Quantidade_4S    0.076717
Lag_Quantidade_1S           0.068478
decoracao                   0.060901
Mes                         0.060555
MediaMovel_Quantidade_8S    0.052029
SemanaDoAno                 0.035472
Lag_Quantidade_2S           0.034125
Lag_Quantidade_3S           0.033924
feriado                     0.025909
instagram                   0.022392
preco_medio_venda           0.000000
Ano                         0.000000
dtype: float32

     *** EXPORTANDO DADOS DE TESTE PARA CSV *** 
Arquivo 'resultados_teste_xgboost_Petisco_Natural_de_Frango_500g.csv' com resultados do teste exportado com sucesso!


In [53]:
# --- 9. PREVISÃO PARA SEMANAS FUTURAS ---
print(f"\nRealizando previsões para as próximas {NUM_FUTURE_WEEKS} semanas (previsão base sem eventos simulados)...")

last_historical_date = df_product_processed.index.max()
future_dates = pd.date_range(start=last_historical_date + pd.Timedelta(weeks=1),
                            periods=NUM_FUTURE_WEEKS, freq='W-MON')

df_combined_for_features = df_product_processed.copy()

future_predictions_list = []
lower_bounds_list = []
upper_bounds_list = []

# Loop para prever cada semana futura, atualizando as features recursivamente
for i in range(NUM_FUTURE_WEEKS):
    current_future_date = future_dates[i]
    
    # 1. Cria um esqueleto da linha futura com base na última semana histórica ou padrão
    temp_future_row = pd.DataFrame(index=[current_future_date])
    temp_future_row['nome_produto'] = PRODUCT_NAME
    temp_future_row['quantidade_real'] = 0 # Placeholder para a previsão
    
    # Preenche colunas opcionais para o futuro:
    # Flags de campanhas/feriados ficam em 0 por padrão, sem simulação aqui.
    # Outros valores como preco_medio_venda e custo_total_campanhas usam o último valor conhecido.
    for col, default_val in optional_cols.items():
        if col in df_product_processed.columns:
            if col in ['decoracao', 'instagram', 'panfletos', 'feriado', 'pessoafisica', 'empresa']:
                temp_future_row[col] = 0 # Mantém em 0 para a previsão base
            elif col in ['preco_medio_venda', 'custo_total_campanhas']:
                # Usa o último valor conhecido da série histórica para estas colunas
                temp_future_row[col] = df_product_processed[col].iloc[-1] if not df_product_processed[col].empty else default_val
            else: 
                temp_future_row[col] = df_product_processed[col].iloc[-1] if not df_product_processed[col].empty else default_val
        else:
            temp_future_row[col] = default_val

    # 2. Adiciona a semana futura temporariamente ao dataframe combinado
    df_combined_for_features = pd.concat([df_combined_for_features, temp_future_row])
    
    # 3. Recalcula as features para TODO o dataframe combinado (para pegar lags/MM corretos para a semana atual)
    # Importante: Criamos um DF temporário com a última linha para gerar features apenas para ela
    X_current_future_features = create_time_features(df_combined_for_features.tail(1)) 
    
    # Garante que as colunas da feature estejam na ordem correta do treinamento
    X_current_future = X_current_future_features.reindex(columns=feature_columns, fill_value=0)

    # 4. Faz a previsão
    pred_raw = model.predict(X_current_future)
    
    # Ajuste para prever zero e arredondar
    pred_value = np.round(pred_raw[0]).clip(min=0)
    if pred_value < 0.5:
        pred_value = 0
    else:
        pred_value = int(pred_value)

    # Margem de erro baseada no MAE do teste (reusando o 'mae' calculado na seção 8)
    # Usamos np.nan_to_num para garantir que não haja NaN se mae for NaN
    error_margin_for_bounds = mae if 'mae' in locals() and not np.isnan(mae) else (y_train.std() * 0.5 if y_train.std() > 0 else 0)

    lower_bound_raw = pred_raw[0] - error_margin_for_bounds
    upper_bound_raw = pred_raw[0] + error_margin_for_bounds
    
    lower_bound = np.round(lower_bound_raw).clip(min=0)
    if lower_bound < 0.5:
        lower_bound = 0
    else:
        lower_bound = int(lower_bound)

    upper_bound = np.round(upper_bound_raw).clip(min=0)
    if upper_bound < 0.5:
        upper_bound = 0
    else:
        upper_bound = int(upper_bound)

    future_predictions_list.append(pred_value)
    lower_bounds_list.append(lower_bound)
    upper_bounds_list.append(upper_bound)

    # 5. Atualiza 'quantidade_real' no DataFrame combinado com a previsão para a próxima iteração
    df_combined_for_features.loc[current_future_date, 'quantidade_real'] = pred_value

print("Previsões futuras (base sem eventos simulados) geradas.")


Realizando previsões para as próximas 12 semanas (previsão base sem eventos simulados)...
Previsões futuras (base sem eventos simulados) geradas.


In [54]:
# --- 10. CONSTRUIR DATAFRAME FINAL PARA EXPORTAÇÃO ---
print("\nMontando DataFrame final para exportação...")

df_final_export = df_product_processed[['quantidade_real']].copy()
df_final_export.rename(columns={'quantidade_real': 'Valores reais (divisao de validacao)'}, inplace=True)
df_final_export['nome_produto'] = PRODUCT_NAME
df_final_export['Valores preditos'] = np.nan
df_final_export['Valores de previsao'] = np.nan
df_final_export['Lower_Bound'] = np.nan
df_final_export['Upper_Bound'] = np.nan

# Popula os valores preditos no período de teste
df_final_export.loc[y_test.index, 'Valores preditos'] = y_pred_test

# Adiciona as previsões futuras e seus intervalos
df_future_results = pd.DataFrame({
    'Valores de previsao': future_predictions_list,
    'Lower_Bound': lower_bounds_list,
    'Upper_Bound': upper_bounds_list
}, index=future_dates)
df_future_results['nome_produto'] = PRODUCT_NAME

# Concatena o histórico com as previsões futuras
df_final_export = pd.concat([df_final_export, df_future_results], axis=0, ignore_index=False)
df_final_export = df_final_export.sort_index()

output_columns_order = [
    'Timestamp',
    'nome_produto',
    'Valores reais (divisao de validacao)',
    'Valores preditos',
    'Valores de previsao',
    'Lower_Bound',
    'Upper_Bound'
]
df_final_export['Timestamp'] = df_final_export.index.strftime('%d/%m/%Y')

for col in output_columns_order:
    if col not in df_final_export.columns:
        df_final_export[col] = np.nan

df_final_export = df_final_export[output_columns_order]

# Formata as colunas numéricas para string com duas casas decimais ou inteiro se for exato
for col in ['Valores reais (divisao de validacao)', 'Valores preditos', 'Valores de previsao', 'Lower_Bound', 'Upper_Bound']:
    df_final_export[col] = df_final_export[col].astype(float).apply(
        lambda x: f"{int(x)}".replace('.', ',') if pd.notna(x) and x == int(x) else (f"{x:.2f}".replace('.', ',') if pd.notna(x) else '')
    )

# --- 11. EXPORTAR O DATAFRAME FINAL PARA CSV ---
safe_product_name = unicodedata.normalize('NFKD', PRODUCT_NAME).encode('ascii', 'ignore').decode('utf-8')
safe_product_name = safe_product_name.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "")
output_filename = f'previsoes_demanda_xgboost_{safe_product_name}.csv'

df_final_export.to_csv(output_filename, index=False, encoding='utf-8-sig', sep=';')

print(f"\nArquivo '{output_filename}' exportado com sucesso!")

# --- 12. CÁLCULO DE IMPACTO PERCENTUAL DE CAMPANHAS E FERIADO (SIMULADO) ---
print("\n" + "="*50)
print("     *** IMPACTO ESTIMADO DE EVENTOS (Cenários Simulados) *** ")
print("==================================================")

# Para calcular o impacto, vamos criar a "base_prediction_row"
# simulando a primeira semana futura sem NENHUM evento/campanha ativado,
# e então ativaremos cada feature para ver o impacto.

# Cria uma linha de features para a primeira semana futura sem NENHUM evento/campanha
first_future_date = future_dates[0]
temp_base_future_row = pd.DataFrame(index=[first_future_date])
temp_base_future_row['nome_produto'] = PRODUCT_NAME
temp_base_future_row['quantidade_real'] = 0 # Não importa para gerar features, é um placeholder

for col, default_val in optional_cols.items():
    if col in df_product_processed.columns:
        if col in ['decoracao', 'instagram', 'panfletos', 'feriado', 'pessoafisica', 'empresa']:
            temp_base_future_row[col] = 0 # Garante que essas flags são 0 para a BASE
        elif col in ['preco_medio_venda', 'custo_total_campanhas']:
            temp_base_future_row[col] = df_product_processed[col].iloc[-1] if not df_product_processed[col].empty else default_val
        else:
            temp_base_future_row[col] = df_product_processed[col].iloc[-1] if not df_product_processed[col].empty else default_val
    else:
        temp_base_future_row[col] = default_val

# Gera as features para esta "linha base" futura
base_future_features = create_time_features(pd.concat([df_product_processed, temp_base_future_row]))

# A base_prediction_row é a última linha das features geradas (correspondente a first_future_date)
base_prediction_row_for_model = base_future_features.tail(1).reindex(columns=feature_columns, fill_value=0)

# A previsão base é a previsão da primeira semana futura sem eventos simulados
base_prediction_value_raw = model.predict(base_prediction_row_for_model)[0]
# Usamos o valor bruto para calcular o impacto percentual
base_prediction_value_for_impact = base_prediction_value_raw

# Função auxiliar para calcular e imprimir o impacto (AGORA COM VALORES DECIMAIS NO CÁLCULO)
def calculate_and_print_impact_decimal(feature_name, simulated_value_feature):
    temp_simulated_row = base_prediction_row_for_model.copy()
    temp_simulated_row.loc[:, feature_name] = simulated_value_feature # Usa .loc para evitar SettingWithCopyWarning
    
    simulated_prediction_value_raw = model.predict(temp_simulated_row)[0]
    
    # Valores para exibição arredondados para inteiros, como você venderia
    base_display = np.round(base_prediction_value_raw).clip(min=0).astype(int)
    simulated_display = np.round(simulated_prediction_value_raw).clip(min=0).astype(int)

    # Cálculo do impacto percentual usando valores brutos (decimais)
    if base_prediction_value_for_impact == 0:
        if simulated_prediction_value_raw > 0:
            impact_str = "Infinito (de 0 para >0)"
        else:
            impact_str = "0.00%" # Continua zero
    else:
        impact_percent = ((simulated_prediction_value_raw - base_prediction_value_for_impact) / base_prediction_value_for_impact) * 100
        impact_str = f"{impact_percent:.2f}%"

    print(f"- {feature_name.replace('_', ' ').title()}: Impacto de {impact_str} (Base Bruta: {base_prediction_value_for_impact:.2f} -> Simulada Bruta: {simulated_prediction_value_raw:.2f} | Arredondado: {base_display} -> {simulated_display})")

# Simulações de impacto
calculate_and_print_impact_decimal('feriado', 1)
calculate_and_print_impact_decimal('instagram', 1)
calculate_and_print_impact_decimal('decoracao', 1)
calculate_and_print_impact_decimal('panfletos', 1)

print("==================================================")
print("\n--- FIM DO ALGORITMO ---")


Montando DataFrame final para exportação...

Arquivo 'previsoes_demanda_xgboost_Petisco_Natural_de_Frango_500g.csv' exportado com sucesso!

     *** IMPACTO ESTIMADO DE EVENTOS (Cenários Simulados) *** 
- Feriado: Impacto de 0.00% (Base Bruta: 1.58 -> Simulada Bruta: 1.58 | Arredondado: 2 -> 2)
- Instagram: Impacto de 1.44% (Base Bruta: 1.58 -> Simulada Bruta: 1.60 | Arredondado: 2 -> 2)
- Decoracao: Impacto de 9.93% (Base Bruta: 1.58 -> Simulada Bruta: 1.73 | Arredondado: 2 -> 2)
- Panfletos: Impacto de 3.66% (Base Bruta: 1.58 -> Simulada Bruta: 1.63 | Arredondado: 2 -> 2)

--- FIM DO ALGORITMO ---


In [55]:
# --- 13. GERAR LINHA DE RESUMO AUTOMATIZADA ---
print("\n" + "="*50)
print("     *** LINHA DE RESUMO AUTOMATIZADA *** ")
print("==================================================")

# Coletando os valores que já estão disponíveis das etapas anteriores
product_name_for_summary = PRODUCT_NAME # Já definido no início do script

# Formatando r2, mae e mape diretamente de suas variáveis numéricas
r_squared_summary = f"{r2:.4f}" if not pd.isna(r2) else "N/A"
mae_summary = f"{mae:.2f}" if not pd.isna(mae) else "N/A"
mape_summary = f"{mape:.2f}%" if not pd.isna(mape) else "N/A" # Usamos 'mape' (numérico) e adicionamos '%' aqui


# Capturando e formatando os impactos percentuais do dicionário 'impact_results' da Etapa 12
# Esta parte não precisa de alteração se 'impact_results' já está sendo preenchido na Etapa 12.
feriado_impact = f"{impact_results.get('feriado', 0.0):.2f}%" if not pd.isna(impact_results.get('feriado')) and impact_results.get('feriado') != float('inf') else "Infinito" if impact_results.get('feriado') == float('inf') else "0.00%"
instagram_impact = f"{impact_results.get('instagram', 0.0):.2f}%" if not pd.isna(impact_results.get('instagram')) and impact_results.get('instagram') != float('inf') else "Infinito" if impact_results.get('instagram') == float('inf') else "0.00%"
decoracao_impact = f"{impact_results.get('decoracao', 0.0):.2f}%" if not pd.isna(impact_results.get('decoracao')) and impact_results.get('decoracao') != float('inf') else "Infinito" if impact_results.get('decoracao') == float('inf') else "0.00%"
panfletos_impact = f"{impact_results.get('panfletos', 0.0):.2f}%" if not pd.isna(impact_results.get('panfletos')) and impact_results.get('panfletos') != float('inf') else "Infinito" if impact_results.get('panfletos') == float('inf') else "0.00%"


# Imprime o cabeçalho da tabela
print("Produto\tR-squared (R²)\tMAE\tMAPE\tFeriado\tInstagram\tDecoração\tPanfletos")
# Imprime a linha de resultados para o produto atual
print(f"{product_name_for_summary}\t{r_squared_summary}\t{mae_summary}\t{mape_summary}\t{feriado_impact}\t{instagram_impact}\t{decoracao_impact}\t{panfletos_impact}")

print("==================================================")


     *** LINHA DE RESUMO AUTOMATIZADA *** 
Produto	R-squared (R²)	MAE	MAPE	Feriado	Instagram	Decoração	Panfletos
Petisco Natural de Frango 500g	0.7619	0.67	17.82%	-5.04%	18.47%	-1.77%	5.41%


In [56]:
# --- 14. EXPORTAR LINHA DE RESUMO PARA CSV ---
print("\n" + "="*50)
print("     *** EXPORTANDO LINHA DE RESUMO PARA CSV *** ")
print("==================================================")

# Dados da linha de resumo (usando as variáveis já definidas na Etapa 13)
summary_data = {
    'Produto': [product_name_for_summary],
    'R-squared (R²)': [r_squared_summary],
    'MAE': [mae_summary],
    'MAPE': [mape_summary],
    'Feriado': [feriado_impact],
    'Instagram': [instagram_impact],
    'Decoração': [decoracao_impact],
    'Panfletos': [panfletos_impact]
}

# Criar DataFrame a partir dos dados
df_summary = pd.DataFrame(summary_data)

# Nome do arquivo de saída. Adicionaremos o nome do produto para cada linha ser única.
safe_product_name = unicodedata.normalize('NFKD', PRODUCT_NAME).encode('ascii', 'ignore').decode('utf-8')
safe_product_name = safe_product_name.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "")
summary_output_filename = f'resumo_metrics_impact_{safe_product_name}.csv'

# Exportar para CSV
# Usamos 'a' (append) para adicionar linhas a um arquivo existente, e 'header=False'
# para não reescrever o cabeçalho após a primeira execução, se o arquivo já existir.
# Se o arquivo não existir, ele será criado e o cabeçalho será escrito na primeira vez.
# Isso exige um pequeno ajuste para a primeira linha, veja a nota abaixo.
df_summary.to_csv(summary_output_filename, mode='a', header=not os.path.exists(summary_output_filename), index=False, encoding='utf-8-sig', sep=';')

print(f"Linha de resumo exportada para '{summary_output_filename}' com sucesso!")
print("==================================================")


     *** EXPORTANDO LINHA DE RESUMO PARA CSV *** 
Linha de resumo exportada para 'resumo_metrics_impact_Petisco_Natural_de_Frango_500g.csv' com sucesso!
